In [1]:
import pandas as pd 
import datetime
import string
import re
import numpy as np
import itertools 

In [2]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',-6)

C:\Users\ibrahim\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
df =pd.read_excel("Data set of Foursqare.xlsx")


In [4]:
male=pd.read_excel('C:/Users/ibrahim/Desktop/Gp1/gender/male/male.xlsx')
female=pd.read_excel('C:/Users/ibrahim/Desktop/Gp1/gender/female/female.xlsx')

In [8]:
df.replace(to_replace=['None'], value=np.nan, inplace=True)

In [9]:
df[df['lastvote']=='None']

,Unnamed: 0,resturant,userName,Date,lastvote,text,Photo,test,rate,countOfrate


In [10]:

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)# Remove the punctuations
    return text.translate(translator)

In [11]:

def remove_emoji(string):

    emoji_pattern = re.compile("["

                           u"\U0001F600-\U0001F64F"  # emoticons

                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs

                           u"\U0001F680-\U0001F6FF"  # transport & map symbols

                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)

                           u"\U00002702-\U000027B0"

                           u"\U000024C2-\U0001F251"

                           "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', string)

In [12]:
def cleanName(Name):
    
    if(Name.find(" ")!=-1):
        Name=Name[:Name.find(" ")] # To remove the last name
    Name=remove_punctuations(Name) # Remove the punctuations and emoji from name
    Name=remove_emoji(Name)
    return Name.lower()

In [13]:
df.head()

,Unnamed: 0,resturant,userName,Date,lastvote,text,Photo,test,rate,countOfrate
0,0,ماما نوره,Yazeed Althobaiti,"July 19, 2019",Upvoted 1 day ago,Quality of shawarma couldn’t be found anywhere else 😍,NaN,['shawarma'],9.0/10,535
1,1,ماما نوره,S A,"July 15, 2019",Upvoted 2 days ago,Best shawarma in Riyadh but very crowded,NaN,"['shawarma in Riyadh', 'crowded']",9.0/10,535
2,2,ماما نوره,Abdulrahman M,"July 12, 2019",Upvoted 1 day ago,The one & only SHAWARMA.,NaN,['SHAWARMA'],9.0/10,535
3,3,ماما نوره,Fooder,"July 15, 2019",Upvoted 1 week ago,Chicken shawarma & falafel,NaN,"['Chicken shawarma', 'falafel']",9.0/10,535
4,4,ماما نوره,Faleh Alsuhaimi,"September 5, 2019",Upvoted 2 weeks ago,Finally ( Family section ). Bigger branch with same great quality. Definitely shawarma is one of the best in Riyadh.,NaN,"['Family section', 'shawarma']",9.0/10,535


In [14]:
def convertTotime(Date): #normalize different form of Date to one form such as 2020-01-01 
    try:
        
        Date=pd.to_datetime(Date, format='%B %d, %Y')
        Date=Date.strftime("%d-%m-%Y")
        
    except:  
        try:
            Date=pd.to_datetime(Date, format='%b %d, %Y')
            Date=Date.strftime("%d-%m-%Y")
        except:
            try:

                Date=pd.to_datetime(Date, format='%B %d')
                Date=Date.strftime("%d-%m-%Y")
                Date=Date.replace('1900','2020')


            except:  
                try:
                    Date=pd.to_datetime(Date, format='%b %d')
                    Date=Date.strftime("%d-%m-%Y")
                    Date=Date.replace('1900','2020')
                except:
                    return Date  
    return Date

In [15]:
#Update the Date to last vote (if there)
def cleanLastvote(lastvote):
    Date=datetime.datetime.now()
    
    if (lastvote=="None"):
        return np.nan
    try:
        if(lastvote.find("week") or lastvote.find("weeks"))!=-1:
            number=re.findall(r'\d+', lastvote)
            Date = datetime.datetime.now() - datetime.timedelta(weeks=int(number[0]))
            return Date.strftime("%d-%m-%Y")

        elif(lastvote.find("day") or lastvote.find("days"))!=-1:
            number=re.findall(r'\d+', lastvote)
            Date = datetime.datetime.now() - datetime.timedelta(days=int(number[0]))
            return Date.strftime("%d-%m-%Y")

        elif(lastvote.find("hour") or lastvote.find("hours"))!=-1:
            number=re.findall(r'\d+', lastvote)
            Date = datetime.datetime.now() - datetime.timedelta(hours=int(number[0])) 
            return Date.strftime("%d-%m-%Y")
        
        elif(lastvote.find("minutes") or lastvote.find("minute"))!=-1:
            number=re.findall(r'\d+', lastvote)
            Date = datetime.datetime.now() - datetime.timedelta(hours=int(number[0])) 
            return Date.strftime("%d-%m-%Y")

        elif(lastvote.find("Upvoted")!=-1):
            return lastvote.replace('Upvoted ',"")

        elif(lastvote.find("Downvoted")!=-1):
            return lastvote.replace('Downvoted ',"")
    except:
        return lastvote
     
    return lastvote

  

In [16]:
#convert the number of day to name
def Days(day):
    Days={0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
    
    return Days[day]

In [17]:
 print(df['lastvote'])

0       Upvoted 1 day ago     
1       Upvoted 2 days ago    
2       Upvoted 1 day ago     
3       Upvoted 1 week ago    
4       Upvoted 2 weeks ago   
5       Upvoted Jan 22        
6       Upvoted 6 days ago    
7       NaN                   
8       NaN                   
9       Upvoted Jan 5         
10      Upvoted Jan 21        
11      NaN                   
12      NaN                   
13      NaN                   
14      NaN                   
15      NaN                   
16      NaN                   
17      Upvoted 2 weeks ago   
18      NaN                   
19      NaN                   
20      NaN                   
21      Upvoted Feb 8         
22      NaN                   
23      NaN                   
24      NaN                   
25      NaN                   
26      NaN                   
27      NaN                   
28      NaN                   
29      NaN                   
30      NaN                   
31      NaN                   
32      

In [18]:
print(df['Date'])

0       July 19, 2019     
1       July 15, 2019     
2       July 12, 2019     
3       July 15, 2019     
4       September 5, 2019 
5       August 16, 2019   
6       October 12, 2019  
7       July 29, 2019     
8       August 17, 2019   
9       August 28, 2019   
10      December 30, 2019 
11      July 25, 2019     
12      October 2, 2019   
13      July 29, 2019     
14      2 weeks ago       
15      November 10, 2019 
16      January 23        
17      October 12, 2019  
18      October 16, 2019  
19      January 20        
20      February 1        
21      January 13        
22      July 30, 2019     
23      July 30, 2019     
24      August 31, 2019   
25      July 28, 2019     
26      November 27, 2019 
27      August 24, 2019   
28      January 13        
29      December 24, 2019 
30      August 24, 2019   
31      December 14, 2019 
32      December 1, 2019  
33      August 24, 2019   
34      September 6, 2019 
35      October 16, 2019  
36      September 6, 2019 
3

In [19]:
df['Date']=df['lastvote'].fillna(df["Date"])

In [20]:
print(df['Date'])

0       Upvoted 1 day ago     
1       Upvoted 2 days ago    
2       Upvoted 1 day ago     
3       Upvoted 1 week ago    
4       Upvoted 2 weeks ago   
5       Upvoted Jan 22        
6       Upvoted 6 days ago    
7       July 29, 2019         
8       August 17, 2019       
9       Upvoted Jan 5         
10      Upvoted Jan 21        
11      July 25, 2019         
12      October 2, 2019       
13      July 29, 2019         
14      2 weeks ago           
15      November 10, 2019     
16      January 23            
17      Upvoted 2 weeks ago   
18      October 16, 2019      
19      January 20            
20      February 1            
21      Upvoted Feb 8         
22      July 30, 2019         
23      July 30, 2019         
24      August 31, 2019       
25      July 28, 2019         
26      November 27, 2019     
27      August 24, 2019       
28      January 13            
29      December 24, 2019     
30      August 24, 2019       
31      December 14, 2019     
32      

In [21]:
df['Date']=df['Date'].apply(lambda x: cleanLastvote(x))

In [22]:

df['Date']=df['Date'].apply(lambda x: convertTotime(x))

In [23]:
df['Date']=pd.to_datetime(df['Date'])#convert type from string to date

In [24]:
df['Day']=df['Date'].dt.dayofweek #covert the Date to Day and store in column 'Day'

In [25]:
df.head()

,Unnamed: 0,resturant,userName,Date,lastvote,text,Photo,test,rate,countOfrate,Day
0,0,ماما نوره,Yazeed Althobaiti,2020-11-03,Upvoted 1 day ago,Quality of shawarma couldn’t be found anywhere else 😍,NaN,['shawarma'],9.0/10,535,1
1,1,ماما نوره,S A,2020-10-03,Upvoted 2 days ago,Best shawarma in Riyadh but very crowded,NaN,"['shawarma in Riyadh', 'crowded']",9.0/10,535,5
2,2,ماما نوره,Abdulrahman M,2020-11-03,Upvoted 1 day ago,The one & only SHAWARMA.,NaN,['SHAWARMA'],9.0/10,535,1
3,3,ماما نوره,Fooder,2020-05-03,Upvoted 1 week ago,Chicken shawarma & falafel,NaN,"['Chicken shawarma', 'falafel']",9.0/10,535,6
4,4,ماما نوره,Faleh Alsuhaimi,2020-02-27,Upvoted 2 weeks ago,Finally ( Family section ). Bigger branch with same great quality. Definitely shawarma is one of the best in Riyadh.,NaN,"['Family section', 'shawarma']",9.0/10,535,3


In [26]:
df['Day']=df['Day'].apply(lambda x:Days(x))#This function converte the number of day to name

In [27]:
df[df['Day'].isnull()!=False]

,Unnamed: 0,resturant,userName,Date,lastvote,text,Photo,test,rate,countOfrate,Day


In [28]:
df.drop(columns="lastvote",inplace=True) #drop lastvote

In [29]:
df[df['userName'].isnull()==True]

,Unnamed: 0,resturant,userName,Date,text,Photo,test,rate,countOfrate,Day


In [30]:
df['userName'].fillna('',inplace=True) #convert all user Name to String

In [31]:
df['clean Name']=df['userName'].apply(lambda x :cleanName(x))

In [32]:
df['clean Name']

0       yazeed                  
1       s                       
2       abdulrahman             
3       fooder                  
4       faleh                   
5       mishary                 
6       sadeem                  
7       aalmarshad              
8       hod                     
9       hanan                   
10      faisal                  
11      bayan                   
12      mr                      
13      القعقاع                 
14      sondos                  
15      malak                   
16      a7mad                   
17      im                      
18      abdulkarim              
19      fares                   
20      kemo                    
21      mohammed                
22      abduulah                
23      ذيب                     
24      maq                     
25      azzam                   
26      shawarma                
27      mohammed                
28      khulood                 
29      lina                    
30      ha

In [33]:
df['clean Name'].value_counts()

abdullah                    42
abdulaziz                   38
mohammed                    37
fooder                      22
faisal                      19
abdulrahman                 17
ali                         16
ahmed                       14
fahad                       14
ahmad                       14
sarah                       14
khaled                      13
a                           13
naif                        12
reem                        11
mohamed                     11
khalid                      11
saad                        11
majed                       10
turki                       9 
sara                        9 
saleh                       9 
mohammad                    8 
nawaf                       8 
wael                        8 
m                           8 
t                           7 
omar                        7 
عبدالله                     7 
sultan                      7 
aziz                        7 
haifa                       7 
s       

In [34]:
male=male.name.tolist()
female=female.name.tolist()

In [35]:
df['gender']=df['clean Name'].apply(lambda x : 'M' if x in male else "F" if x in female else "Known") #Determain the gender

In [36]:
df[df['gender']=="Known"][['gender','clean Name']].count()

gender        320
clean Name    320
dtype: int64

In [37]:
df[df['gender']=="Known"]['clean Name'].value_counts()

a                           13
m                           8 
s                           7 
t                           7 
n                           7 
b                           6 
                            5 
dr                          5 
al                          5 
mr                          4 
the                         4 
mani                        3 
speed                       3 
القعقاع                     3 
khaledbinsuwaidan           3 
gökhan                      2 
shabeel                     2 
masuod                      2 
arzu                        2 
erol                        2 
jamel                       2 
haluk                       2 
roshaan                     2 
thabit                      2 
r                           2 
aalmarshad                  2 
nadine                      2 
asif                        2 
sercan                      2 
eng                         2 
h                           2 
abdulmuhsen                 2 
bala    

In [38]:
df.loc[20000:20200]

,Unnamed: 0,resturant,userName,Date,text,Photo,test,rate,countOfrate,Day,clean Name,gender


In [39]:
#Convert list of String to list of list
def convertlist(li):
    res = li.strip('][').split(', ') 
    newli=[]
    for r in res:
        newli.append(r.replace("'",""))
    return newli

In [40]:
df.test=df.test.apply(lambda x : convertlist(x))

In [41]:
def lowercase(li):
    nl=[]
    for i in li:
        i=i.lower()
        nl.append(i)
        
    return nl

In [42]:
df.test=df.test.apply(lambda x :lowercase(x))

In [44]:
df['resturant']=df['resturant'].str.replace("bf burger boutique","burger boutique")
df['resturant']=df['resturant'].str.replace("p f changs","pf changs")

In [45]:
df['resturant'].value_counts()

ماما نوره    1294
ويتش وتش     184 
Name: resturant, dtype: int64

In [47]:
df["Date"]=df["Date"].astype(str)

In [48]:
df.head()

,Unnamed: 0,resturant,userName,Date,text,Photo,test,rate,countOfrate,Day,clean Name,gender
0,0,ماما نوره,Yazeed Althobaiti,2020-11-03,Quality of shawarma couldn’t be found anywhere else 😍,NaN,[shawarma],9.0/10,535,Tue,yazeed,M
1,1,ماما نوره,S A,2020-10-03,Best shawarma in Riyadh but very crowded,NaN,"[shawarma in riyadh, crowded]",9.0/10,535,Sat,s,Known
2,2,ماما نوره,Abdulrahman M,2020-11-03,The one & only SHAWARMA.,NaN,[shawarma],9.0/10,535,Tue,abdulrahman,M
3,3,ماما نوره,Fooder,2020-05-03,Chicken shawarma & falafel,NaN,"[chicken shawarma, falafel]",9.0/10,535,Sun,fooder,M
4,4,ماما نوره,Faleh Alsuhaimi,2020-02-27,Finally ( Family section ). Bigger branch with same great quality. Definitely shawarma is one of the best in Riyadh.,NaN,"[family section, shawarma]",9.0/10,535,Thu,faleh,M


In [49]:
resturants=set(df.resturant)

In [50]:
for i in resturants:
    print(i)

ماما نوره
ويتش وتش


In [52]:
for rest in resturants:
    print(rest)
    li=df[df["resturant"]==rest]["test"]
    test=pd.DataFrame(itertools.chain.from_iterable(li),columns={"type"})
    
    test.to_excel("test\ "+rest+".xlsx")
   


ماما نوره
ويتش وتش
